In [1]:
import sys
sys.path.insert(0, './src')

# Import libs

In [2]:
import os
from pathlib import Path

import pandas as pd


from utils import dataset_processed
from next_token_dataset import create_train_val_test_splits

BASE_DIR = Path().resolve()
MAX_LEN = 140


# 1. Clean raw data and tokenize dataset

In [3]:
dataset_processed(
    os.path.join(BASE_DIR, 'data', 'tweets.txt'),
    os.path.join(BASE_DIR, 'data', 'cleaned_tweets.csv'),
    os.path.join(BASE_DIR, 'data', 'tokens_cleaned_tweets.csv')
)
dataset = pd.read_csv(os.path.join(BASE_DIR, 'data', 'tokens_cleaned_tweets.csv'), index_col=False)
dataset.head()

,tokens
0,"[['switchfoot', 'awww', 'thats', 'a', 'bummer'..."
1,"[['is', 'upset', 'that', 'he', 'cant', 'update..."
2,"[['kenichan', 'i', 'dived', 'many', 'times', '..."
3,"[['my', 'whole', 'body', 'feels', 'itchy', 'an..."
4,"[['nationwideclass', 'no', 'its', 'not', 'beha..."


# 2. Split dataset by train, val, test

In [4]:
train, val, test, vocab = create_train_val_test_splits(dataset['tokens'], seq_length=MAX_LEN)

In [7]:
train.dataset.get_vocab_size()

45